In [52]:
import numpy as np
import dash_bootstrap_components as dbc
import pandas as pd
from dash import  dcc, Input, Output, html
import dash
import plotly.express as px

In [53]:
data = pd.read_csv('stockmetadata_Abhir.csv')

In [54]:
data["IBCompare"] = np.select(
    [data["Company_Name"] == data["IB-longName"], data["Company_Name"] != data["IB-longName"]],
    ["Match", "Mismatch or NA"])

In [55]:
data["EODCompare"] = np.select([data["Ticker"] == data["EOD-Code"], data["Ticker"] != data["EOD-Code"]],
                               ["Match", "Mismatch or NA"])

In [56]:
data["FIGICompare"] = np.select(
    [data["Company_Name"] == data["OpenFigi-OpenFigi-name"], data["Company_Name"] != data["OpenFigi-OpenFigi-name"]],
    ["Match", "Mismatch or NA"])

In [57]:
data["ISINdbCompare"] = np.select([data["IB-ISIN"] == data["isindb-ISIN"], data["IB-ISIN"] != data["isindb-ISIN"]],
                                  ["Match", "Mismatch or NA"])

In [16]:
# data["INVCompare"] = np.select([data["IB-ISIN"] == data["INV_isin"], data["IB-ISIN"] != data["INV_isin"]],
#                               ["Match", "Mismatch or NA"])

In [58]:
ibcheck = data[data["IBCompare"].isin(["Mismatch or NA"])]
eodcheck = data[data["EODCompare"].isin(["Mismatch or NA"])]
figicheck = data[data["FIGICompare"].isin(["Mismatch or NA"])]
isindbcheck = data[data["ISINdbCompare"].isin(["Mismatch or NA"])]

In [59]:
histdf=data.rename(columns={"Exchange":'Exchange-OG','IB-industry':'Industry-IB','OpenFigi-OpenFigi-exchCode':'Exchanges-OpenFIGI','EOD-Exchange':'Exchanges-EOD','EOD-Currency':'Currency-EOD','EOD-Sector':'Sector-EOD','EOD-Industry':'Industry-EOD','EOD-GicSector':'GICSector-EOD','EOD-GicGroup':'GICGroup-EOD','EOD-GicIndustry':'GICIndustry-EOD','EOD-GicSubIndustry':'GICSubIndustry-EOD','INV_currency':'Currency-INV'})

In [60]:
options=['Exchange-OG','Industry-IB','Exchanges-OpenFIGI','Exchanges-EOD','Currency-EOD','Sector-EOD','Industry-EOD','GICSector-EOD','GICGroup-EOD','GICIndustry-EOD','GICSubIndustry-EOD','Currency-INV']

In [70]:
layout = html.Div([
    dbc.Col(html.Center(html.H1('TYPE 2 DASHBOARD'))),
    dbc.Col(dcc.Dropdown(id='dropdown',options = options,value = 'Select a Data Slice!')),
    html.Hr(),
    html.Div([
        dcc.Tabs(id="histtab", value='Select',children=[
            dcc.Tab(label='E', value='E'),
            dcc.Tab(label='S', value='S'),
            dcc.Tab(label='G', value='G'),
            dcc.Tab(label='ESGMean', value='ESG_Mean'),
            dcc.Tab(label='STSE', value='STS_E'),
            dcc.Tab(label='STSS', value='STS_S'),
            dcc.Tab(label='STSG', value='STS_G'),
            dcc.Tab(label='ESGSTSMean', value='ESG_STS_Mean'),
            dcc.Tab(label='LTSE', value='LTS_E'),
            dcc.Tab(label='LTSS', value='LTS_S'),
            dcc.Tab(label='LTSG', value='LTS_G'),
            dcc.Tab(label='ESGLTSMean', value='ESG_LTS_Mean')])]),
    dcc.Graph(id="histgraph"),
    dcc.Dropdown(id='dropdown2'),
    dcc.Graph(id="corrgraph"),
    dcc.Dropdown(id='dropdown3'),
    dcc.Graph(id='spydergraph'),
    html.Hr(),
    dbc.Col(html.Center(html.H1('Company Origins'))),
    # html.Hr(),
    # dcc.Graph(figure=countryfig,style={'background':'Black'}),
    html.Hr(),
    dbc.Col(html.Center(html.H1('Data Source'))),
    html.Hr(),
    dcc.Graph(id="sourcegraph"),
    dcc.RadioItems(id="radioitems",options=['IBCompare', 'EODCompare','FIGICompare','ISINdbCompare'],value='IBCompare',inline=True)])


In [71]:
app = dash.Dash(__name__)
server = app.server

app.layout = layout

In [72]:
@app.callback(
    Output('dropdown2', 'options'),
    [Input('dropdown', 'value')])
def update_dropdown2(dropdown):
    attempt = histdf[dropdown].unique()
    return [{'label': i, 'value': i} for i in attempt]

In [73]:
@app.callback(
    Output('dropdown3', 'options'),
    [Input('dropdown', 'value')])
def update_dropdown2(dropdown):
    attempt = histdf[dropdown].unique()
    return [{'label': i, 'value': i} for i in attempt]

In [74]:
@app.callback(
    Output('histgraph', 'figure'),
    [Input('dropdown', 'value'),
     Input('histtab','value')])

def update_histgraph(dropdown,histtab):
        new=histdf.groupby([dropdown]).mean()
        attempt=new[histtab]
        colors=attempt.index
        fig= px.histogram(x=attempt,y=attempt.index,color=colors)
        fig.update_layout(title_text=f"Average {histtab} Scores for {dropdown}", yaxis_title=dropdown, xaxis_title='',title_x=0.5)
        return fig

In [75]:
@app.callback(Output('corrgraph', 'figure'),
              [Input('dropdown', 'value'),
               Input('dropdown2','value')])
def update_corrgraph(dropdown,dropdown2):
    new=histdf.groupby(dropdown)[['E','S','G','ESG_Mean','STS_E','STS_S','STS_G','ESG_STS_Mean','LTS_E','LTS_S','LTS_G','ESG_LTS_Mean']].corr().loc[dropdown2]
    fig = px.imshow(new,color_continuous_scale="blackbody")
    fig.update_layout(title_text=f"Correlation Heatmap for {dropdown2} from {dropdown}",title_x=0.5)
    return fig

In [76]:
@app.callback(Output('spydergraph', 'figure'),
              [Input('dropdown', 'value'),
               Input('dropdown3','value')])
def update_sypdergraph(dropdown,dropdown3):
    new=histdf.groupby([dropdown]).sum()
    new=new.iloc[:,:12]
    attempt=new.loc[dropdown3]
    fig = px.line_polar(attempt, r=attempt, theta=attempt.index, line_close=True)
    fig.update_layout(title_text=f"Total Scores for {dropdown3} from {dropdown}",title_x=0.5)
    return fig

In [77]:
@app.callback(Output('sourcegraph', 'figure'),
              [Input('radioitems', 'value')])
def update_sourcegraph(input_value):
    fig=px.histogram(data,x=input_value,color=input_value)
    fig.update_layout(title_text=f"{input_value}",title_x=0.5)
    return fig

In [78]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Aug/2022 16:52:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2022 16:52:09] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2022 16:52:09] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2022 16:52:09] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2022 16:52:09] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2022 16:52:09] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/opt/anaconda3/lib/python3.9/site-packages/dash/dash.py", line 1259, in dispatch
    ctx.run(
  Fil

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/opt/anaconda3/lib/python3.9/site-packages/dash/dash.py", line 1259, in dispatch
    ctx.run(
  Fil

127.0.0.1 - - [17/Aug/2022 16:52:09] "

...

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3621, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 136, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 163, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: None

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle

127.0.0.1 - - [17/Aug/2022 16:52:13] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/opt/anaconda3/lib/python3.9/site-packages/dash/dash.py", line 1259, in dispatch
    ctx.run(
  Fil

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/opt/anaconda3/lib/python3.9/site-packages/dash/dash.py", line 1259, in dispatch
    ctx.run(
  Fil

127.0.0.1 - - [17/Aug/2022 16:52:13] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [17/Aug/2022 16:52:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2022 16:52:17] "POST /_dash-update-component HTTP/1.1" 200 -
/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

127.0.0.1 - - [17/Aug/2022 16:52:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2022 16:52:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2022 16:52:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2022 16:52:25] "POST /_dash-update-component HTTP/1.1" 200 -
